# 학습된 모델 테스트 및 평가 파이프라인

이 노트북은 학습 완료된 두 모델을 사용하여 실제 데이터로 테스트합니다:
- **U-Net 모델** (Federated_learning_merged_data.ipynb에서 학습): 결함 검출
- **CNN 모델** (CNN_Federated_Learning.ipynb에서 학습): 결함 유형 분류

## 진행 순서
1. **환경 설정**: GPU 확인 및 라이브러리 import
2. **모델 로드**: U-Net과 CNN 모델 로드
3. **데이터 준비**: 테스트 데이터 준비 및 레이블 매핑 생성
4. **U-Net 테스트**: 결함 검출 성능 평가
5. **CNN 테스트**: 결함 유형 분류 성능 평가
6. **결과 시각화**: 검출 및 분류 결과 분석


In [1]:
# GPU 확인
import torch
import tensorflow as tf

# TensorFlow GPU 확인
device_name = tf.test.gpu_device_name()
print('TensorFlow GPU:', 'Found GPU at: {}'.format(device_name) if device_name == '/device:GPU:0' else 'GPU not found.')

# PyTorch GPU 확인
print('PyTorch GPU:', f'Found GPU: {torch.cuda.get_device_name(0)}' if torch.cuda.is_available() else 'GPU not found.')


TensorFlow GPU: GPU not found.
PyTorch GPU: Found GPU: NVIDIA GeForce RTX 3070


In [2]:
# 필요한 라이브러리 import
import numpy as np
import torch
import tensorflow as tf
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# 유틸리티 함수 import
from utils.cnn.defect_detection import scan_data_directory, load_image_for_cnn, get_defect_type_from_npy
from utils.cnn.classifier import load_cnn_model
from utils.cnn.dataset_functions import get_label_mapping, create_cnn_dataset, unwrap_client_data
from utils.u_net.dataset_functions import create_dataset, unwrap_client_data as unwrap_unet_data
from utils.u_net.image_processing import unsplit_image_mask

print("라이브러리 import 완료!")


라이브러리 import 완료!


In [3]:
# 경로 설정
data_dir = 'data'  # data 폴더 경로
unet_model_path = 'saved_models/FL_2_5_32_8e05_HoldoutPart06_1.h5'  # U-Net 모델 경로
cnn_model_path = 'saved_models/CNN_FL_1_1_32_1e04_HoldoutPart06.pth'  # CNN 모델 경로

# 테스트 데이터 설정
max_files = None  # None이면 전체 사용, 숫자를 지정하면 해당 개수만 사용
test_data_ratio = 0.15  # 전체 데이터에서 테스트 데이터 비율

print(f"Data 디렉터리: {data_dir}")
print(f"U-Net 모델 경로: {unet_model_path}")
print(f"CNN 모델 경로: {cnn_model_path}")
print(f"최대 파일 개수: {max_files if max_files else '전체'}")


Data 디렉터리: data
U-Net 모델 경로: saved_models/FL_2_5_32_8e05_HoldoutPart06_1.h5
CNN 모델 경로: saved_models/CNN_FL_1_1_32_1e04_HoldoutPart06.pth
최대 파일 개수: 전체


In [4]:
## 1단계: 모델 로드

print("="*60)
print("1단계: 학습된 모델 로드")
print("="*60)

# U-Net 모델 로드 (TensorFlow)
print("\n[1-1] U-Net 모델 로드 중...")
unet_model = tf.keras.models.load_model(unet_model_path, compile=False)
print("✓ U-Net 모델 로드 완료!")
print(f"  입력 shape: {unet_model.input_shape}")
print(f"  출력 shape: {unet_model.output_shape}")

# CNN 모델 로드 (PyTorch)
print("\n[1-2] CNN 모델 로드 중...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cnn_model = load_cnn_model(cnn_model_path, device=device)
print("✓ CNN 모델 로드 완료!")
print(f"  Device: {device}")
print(f"  모델 파라미터 수: {sum(p.numel() for p in cnn_model.parameters()):,}")

1단계: 학습된 모델 로드

[1-1] U-Net 모델 로드 중...
✓ U-Net 모델 로드 완료!
  입력 shape: (None, 128, 128, 2)
  출력 shape: (None, 128, 128, 3)

[1-2] CNN 모델 로드 중...
✓ CNN 모델 로드 완료!
  Device: cuda
  모델 파라미터 수: 5,110,731


In [5]:
## 2단계: 데이터 준비 및 레이블 매핑 생성

print("\n" + "="*60)
print("2단계: 테스트 데이터 준비")
print("="*60)

# 데이터 디렉터리 스캔
print("\n[2-1] 데이터 디렉터리 스캔 중...")
file_list = scan_data_directory(data_dir)
print(f"✓ 총 {len(file_list)}개의 파일 발견")

# 데이터 개수 제한 적용
if max_files is not None and max_files > 0:
    file_list = file_list[:max_files]
    print(f"✓ 데이터 개수 제한: {len(file_list)}개 파일 사용")

# 테스트 데이터 분할 (CNN 모델 학습 시와 동일한 방식)
import random

random.seed(42)  # 재현성을 위한 시드 설정
test_client_id = 6
num_clients = 8
train_clients = [f'client{i}' for i in range(1, num_clients + 1) if i != test_client_id]

# 전체 파일을 클라이언트에 분배 (CNN 학습 시와 동일한 방식)
random.shuffle(file_list)
files_per_client = len(file_list) // len(train_clients)

clientIdentifierDict = {}
test_files = []

start_idx = 0
for i, client_id in enumerate(train_clients):
    if i < len(train_clients) - 1:
        end_idx = start_idx + files_per_client
    else:
        end_idx = len(file_list)
    
    client_files = file_list[start_idx:end_idx]
    
    # 테스트 데이터 추출
    num_test_from_client = int(len(client_files) * test_data_ratio)
    test_files_from_client = random.sample(client_files, num_test_from_client)
    test_files.extend(test_files_from_client)
    
    # 학습 데이터
    train_files = [f for f in client_files if f not in test_files_from_client]
    clientIdentifierDict[client_id] = train_files
    
    start_idx = end_idx

# 테스트 클라이언트 설정
clientIdentifierDict[f'client{test_client_id}'] = test_files

print(f"\n✓ 클라이언트 분배 완료:")
for client_id, files in clientIdentifierDict.items():
    if client_id == f'client{test_client_id}':
        print(f"  {client_id} (테스트): {len(files)}개 파일")
    else:
        print(f"  {client_id} (학습): {len(files)}개 파일")

# 레이블 매핑 생성 (CNN 모델과 동일한 매핑 필요)
print("\n[2-2] 레이블 매핑 생성 중...")
label_mapping, num_classes = get_label_mapping(data_dir, clientIdentifierDict)
print(f"✓ 레이블 매핑 완료: {num_classes}개 클래스")


2단계: 테스트 데이터 준비

[2-1] 데이터 디렉터리 스캔 중...
✓ 총 693개의 파일 발견

✓ 클라이언트 분배 완료:
  client1 (학습): 85개 파일
  client2 (학습): 85개 파일
  client3 (학습): 85개 파일
  client4 (학습): 85개 파일
  client5 (학습): 85개 파일
  client7 (학습): 85개 파일
  client8 (학습): 85개 파일
  client6 (테스트): 98개 파일

[2-2] 레이블 매핑 생성 중...
발견된 결함 유형 (0과 1 제외): [np.int8(-1), np.uint8(3), np.uint8(4), np.uint8(5), np.uint8(6), np.int8(7), np.uint8(8), np.uint8(9), np.int8(11), np.uint8(14), np.uint8(255)]
레이블 매핑: {np.int8(-1): 0, np.uint8(3): 1, np.uint8(4): 2, np.uint8(5): 3, np.uint8(6): 4, np.int8(7): 5, np.uint8(8): 6, np.uint8(9): 7, np.int8(11): 8, np.uint8(14): 9, np.uint8(255): 10}
총 클래스 수: 11
✓ 레이블 매핑 완료: 11개 클래스


In [6]:
## 3단계: U-Net 모델 테스트 - 결함 검출 성능 평가

print("\n" + "="*60)
print("3단계: U-Net 결함 검출 테스트")
print("="*60)

# U-Net용 데이터셋 생성
tileSize = 128
defect_threshold = 0.01

test_client_id = 6
test_clients = [f'client{test_client_id}']
test_files = clientIdentifierDict[f'client{test_client_id}']

# 테스트 파일 그룹 생성
test_file_groups = {'test': test_files}

imagePath0 = f'{data_dir}/0/'
imagePath1 = f'{data_dir}/1/'
npyPath = f'{data_dir}/annotations/'

print("\n[3-1] U-Net 테스트 데이터셋 생성 중...")
testImageDict, testMaskDict = create_dataset(
    test_file_groups,
    imagePath0,
    imagePath1,
    npyPath,
    tileSize=tileSize
)
print("✓ 데이터셋 생성 완료")

# U-Net 예측 수행
print("\n[3-2] U-Net 예측 수행 중...")
testImages, testMasks = unwrap_unet_data(testImageDict, testMaskDict, ['test'])
print(f"총 {testImages.shape[0]}개 타일 예측 중...")

predictedImages = unet_model.predict(testImages, verbose=0)
predictedMask = tf.argmax(predictedImages, axis=-1)

# 각 이미지별로 결과 수집
unet_results = []
num_tiles_per_image = 25  # 640x640 = 5*5 = 25 타일
curr_idx = 0

for file_name in test_files:
    prev_idx = curr_idx
    curr_idx = prev_idx + num_tiles_per_image
    
    # 타일 재구성
    imageheight, imagewidth = 5*128, 5*128  # 640x640
    fullPredictedMask = unsplit_image_mask(
        predictedMask[prev_idx:curr_idx],
        imageheight,
        imagewidth
    )
    
    # 결함 검출
    defect_mask = fullPredictedMask[0, :, :, 0].numpy()
    defect_ratio = np.sum(defect_mask == 2) / defect_mask.size if defect_mask.size > 0 else 0.0
    has_defect = defect_ratio > defect_threshold
    
    # 실제 레이블 확인
    npy_path = Path(npyPath) / f"{file_name}.npy"
    actual_defect_type, _ = get_defect_type_from_npy(str(npy_path))
    actual_has_defect = actual_defect_type is not None
    
    unet_results.append({
        'file_name': file_name,
        'predicted_has_defect': has_defect,
        'actual_has_defect': actual_has_defect,
        'defect_ratio': defect_ratio,
        'mask': defect_mask
    })
    
    if curr_idx >= len(predictedMask):
        break

# 성능 평가
true_positives = sum(1 for r in unet_results if r['predicted_has_defect'] and r['actual_has_defect'])
false_positives = sum(1 for r in unet_results if r['predicted_has_defect'] and not r['actual_has_defect'])
false_negatives = sum(1 for r in unet_results if not r['predicted_has_defect'] and r['actual_has_defect'])
true_negatives = sum(1 for r in unet_results if not r['predicted_has_defect'] and not r['actual_has_defect'])

unet_precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
unet_recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
unet_f1_score = 2 * (unet_precision * unet_recall) / (unet_precision + unet_recall) if (unet_precision + unet_recall) > 0 else 0
unet_accuracy = (true_positives + true_negatives) / len(unet_results) if len(unet_results) > 0 else 0

print(f"\n✓ U-Net 검출 완료!")
print(f"\n[검출 성능]")
print(f"  정확도 (Accuracy): {unet_accuracy:.4f}")
print(f"  정밀도 (Precision): {unet_precision:.4f}")
print(f"  재현율 (Recall): {unet_recall:.4f}")
print(f"  F1 점수: {unet_f1_score:.4f}")
print(f"\n[검출 결과]")
print(f"  True Positives: {true_positives}")
print(f"  False Positives: {false_positives}")
print(f"  False Negatives: {false_negatives}")
print(f"  True Negatives: {true_negatives}")
print(f"  총 테스트 파일: {len(unet_results)}개")



3단계: U-Net 결함 검출 테스트

[3-1] U-Net 테스트 데이터셋 생성 중...

test...
  처리 중: 50/98 파일 완료 (현재 타일 수: 3050)
  처리 중: 98/98 파일 완료 (현재 타일 수: 5250)
Contains 98 images...
Tiled Image Tensor Shape:  (5250, 128, 128, 2)
Tiled Mask Shape:  (5250, 128, 128)
✓ 데이터셋 생성 완료

[3-2] U-Net 예측 수행 중...
총 5250개 타일 예측 중...

✓ U-Net 검출 완료!

[검출 성능]
  정확도 (Accuracy): 0.3061
  정밀도 (Precision): 0.9565
  재현율 (Recall): 0.2472
  F1 점수: 0.3929

[검출 결과]
  True Positives: 22
  False Positives: 1
  False Negatives: 67
  True Negatives: 8
  총 테스트 파일: 98개


In [ ]:
## 4단계: CNN 모델 테스트 - 결함 유형 분류 성능 평가

print("\n" + "="*60)
print("4단계: CNN 결함 유형 분류 테스트")
print("="*60)

# CNN용 테스트 데이터셋 생성
print("\n[4-1] CNN 테스트 데이터셋 생성 중...")
target_size = (640, 640)
testImageDict, testLabelDict = create_cnn_dataset(
    {f'client{test_client_id}': test_files},
    data_dir,
    target_size=target_size,
    label_mapping=label_mapping
)
print("✓ 데이터셋 생성 완료")

# 테스트 데이터 준비
testImages, testLabels = unwrap_client_data(testImageDict, testLabelDict, [f'client{test_client_id}'])
testImages = testImages.to(device)

print(f"테스트 이미지 shape: {testImages.shape}")
print(f"테스트 레이블 shape: {testLabels.shape}")

# CNN 예측 수행
print("\n[4-2] CNN 예측 수행 중...")
batch_size = 32
predictions = []
probabilities_list = []

cnn_model.eval()
with torch.no_grad():
    for i in range(0, len(testImages), batch_size):
        batch_images = testImages[i:i+batch_size]
        outputs = cnn_model(batch_images)
        probs = torch.softmax(outputs, dim=1)
        preds = torch.argmax(probs, dim=1)
        
        predictions.extend(preds.cpu().numpy())
        probabilities_list.extend(probs.cpu().numpy())

predictions = np.array(predictions)
probabilities_list = np.array(probabilities_list)
test_labels_np = testLabels.numpy()

# 성능 평가
correct = (predictions == test_labels_np).sum()
cnn_accuracy = correct / len(test_labels_np)

print(f"\n✓ CNN 분류 완료!")
print(f"\n[분류 성능]")
print(f"  정확도 (Accuracy): {cnn_accuracy:.4f} ({correct}/{len(test_labels_np)})")

# 실제로 존재하는 클래스만 추출
unique_labels = np.unique(np.concatenate([test_labels_np, predictions]))
unique_labels = np.sort(unique_labels)
print(f"\n  테스트 데이터에 존재하는 클래스: {unique_labels.tolist()}")
print(f"  총 클래스 수: {len(unique_labels)}개 (전체 {num_classes}개 중)")

# 상세 성능 리포트 (실제 존재하는 클래스만 사용)
print(f"\n[상세 분류 리포트]")
target_names = [f'Class_{i}' for i in unique_labels]
print(classification_report(test_labels_np, predictions, 
                          labels=unique_labels,
                          target_names=target_names,
                          zero_division=0))

# Confusion Matrix 생성 (실제 존재하는 클래스만 사용)
cm = confusion_matrix(test_labels_np, predictions, labels=unique_labels)
print(f"\n[Confusion Matrix]")
print(f"클래스: {unique_labels.tolist()}")
print(cm)



4단계: CNN 결함 유형 분류 테스트

[4-1] CNN 테스트 데이터셋 생성 중...
client6...
Contains 98 images...
Valid images: 89 (filtered 9 invalid labels)
Image Tensor Shape: torch.Size([89, 2, 640, 640])
Label Shape: torch.Size([89])
Label distribution: Counter({np.int64(6): 32, np.int64(4): 16, np.int64(3): 13, np.int64(1): 12, np.int64(10): 5, np.int64(8): 4, np.int64(5): 3, np.int64(7): 3, np.int64(0): 1})
✓ 데이터셋 생성 완료
테스트 이미지 shape: torch.Size([89, 2, 640, 640])
테스트 레이블 shape: torch.Size([89])

[4-2] CNN 예측 수행 중...

✓ CNN 분류 완료!

[분류 성능]
  정확도 (Accuracy): 0.0562 (5/89)

[상세 분류 리포트]


ValueError: Number of classes, 9, does not match size of target_names, 11. Try specifying the labels parameter

In [ ]:
## 5단계: 결과 시각화

print("\n" + "="*60)
print("5단계: 결과 시각화")
print("="*60)

# unique_labels가 정의되지 않은 경우를 대비 (Cell 7을 먼저 실행해야 함)
if 'unique_labels' not in locals():
    # 실제로 존재하는 클래스만 추출
    unique_labels = np.unique(np.concatenate([test_labels_np, predictions]))
    unique_labels = np.sort(unique_labels)
    print(f"\n  참고: 테스트 데이터에 존재하는 클래스: {unique_labels.tolist()}")

# 5-1: U-Net 검출 결과 시각화
print("\n[5-1] U-Net 검출 결과 시각화")

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# 결함이 검출된 이미지 몇 개 선택
defect_detected = [r for r in unet_results if r['predicted_has_defect']]
sample_results = defect_detected[:6] if len(defect_detected) >= 6 else defect_detected

for idx, result in enumerate(sample_results):
    if idx >= 6:
        break
    
    file_name = result['file_name']
    img0_path = Path(imagePath0) / f"{file_name}.jpg"
    img1_path = Path(imagePath1) / f"{file_name}.jpg"
    
    # 원본 이미지 로드 (Post Fusion)
    img = plt.imread(str(img1_path))
    axes[idx].imshow(img, cmap='gray')
    axes[idx].set_title(f"{file_name}\n결함 비율: {result['defect_ratio']:.3f}\n"
                       f"예측: {'결함' if result['predicted_has_defect'] else '정상'}\n"
                       f"실제: {'결함' if result['actual_has_defect'] else '정상'}",
                       fontsize=10)
    axes[idx].axis('off')

# 빈 subplot 제거
for idx in range(len(sample_results), 6):
    axes[idx].axis('off')

plt.suptitle('U-Net 결함 검출 결과 (샘플)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# 5-2: CNN 분류 Confusion Matrix 시각화
print("\n[5-2] CNN 분류 Confusion Matrix 시각화")

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=[f'Class_{i}' for i in unique_labels],
            yticklabels=[f'Class_{i}' for i in unique_labels])
plt.title('CNN 결함 유형 분류 Confusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('예측 레이블')
plt.ylabel('실제 레이블')
plt.tight_layout()
plt.show()

# 5-3: 클래스별 정확도 (실제 존재하는 클래스만)
print("\n[5-3] 클래스별 분류 정확도")

class_accuracies = []
for i in unique_labels:
    mask = test_labels_np == i
    if mask.sum() > 0:
        class_acc = (predictions[mask] == test_labels_np[mask]).sum() / mask.sum()
        class_accuracies.append(class_acc)
    else:
        class_accuracies.append(0)

plt.figure(figsize=(12, 6))
bars = plt.bar(range(len(unique_labels)), class_accuracies, color='steelblue', alpha=0.7)
plt.xlabel('클래스', fontsize=12)
plt.ylabel('정확도', fontsize=12)
plt.title('클래스별 분류 정확도', fontsize=14, fontweight='bold')
plt.xticks(range(len(unique_labels)), [f'Class_{i}' for i in unique_labels], rotation=45)
plt.ylim([0, 1.1])
plt.grid(axis='y', alpha=0.3)

# 값 표시
for i, (bar, acc) in enumerate(zip(bars, class_accuracies)):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.02,
             f'{acc:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# 5-4: 예측 확신도 분포
print("\n[5-4] 예측 확신도 분포")

max_probs = probabilities_list.max(axis=1)
correct_probs = probabilities_list[np.arange(len(predictions)), predictions]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 전체 예측 확신도
axes[0].hist(max_probs, bins=30, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].set_xlabel('최대 예측 확률', fontsize=12)
axes[0].set_ylabel('빈도', fontsize=12)
axes[0].set_title('전체 예측 확신도 분포', fontsize=12, fontweight='bold')
axes[0].grid(alpha=0.3)

# 정확/오분류별 확신도
correct_mask = predictions == test_labels_np
axes[1].hist(max_probs[correct_mask], bins=30, alpha=0.7, label='정확', color='green', edgecolor='black')
axes[1].hist(max_probs[~correct_mask], bins=30, alpha=0.7, label='오분류', color='red', edgecolor='black')
axes[1].set_xlabel('최대 예측 확률', fontsize=12)
axes[1].set_ylabel('빈도', fontsize=12)
axes[1].set_title('정확/오분류별 예측 확신도', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ 시각화 완료!")


In [ ]:
## 6단계: 종합 결과 요약

print("\n" + "="*60)
print("6단계: 종합 결과 요약")
print("="*60)

print("\n[U-Net 모델 성능]")
print(f"  모델 경로: {unet_model_path}")
print(f"  결함 검출 정확도: {unet_accuracy:.4f}")
print(f"  정밀도: {unet_precision:.4f}")
print(f"  재현율: {unet_recall:.4f}")
print(f"  F1 점수: {unet_f1_score:.4f}")

print("\n[CNN 모델 성능]")
print(f"  모델 경로: {cnn_model_path}")
print(f"  결함 유형 분류 정확도: {cnn_accuracy:.4f}")
print(f"  총 클래스 수: {num_classes}")
print(f"  테스트 샘플 수: {len(test_labels_np)}")

print("\n[테스트 데이터 정보]")
print(f"  총 테스트 파일 수: {len(test_files)}")
print(f"  U-Net 검출 완료: {len(unet_results)}개")
print(f"  CNN 분류 가능 파일: {len(test_labels_np)}개")

print("\n[모델 정보]")
print(f"  U-Net 입력 shape: {unet_model.input_shape}")
print(f"  U-Net 출력 shape: {unet_model.output_shape}")
print(f"  CNN device: {device}")
print(f"  CNN 파라미터 수: {sum(p.numel() for p in cnn_model.parameters()):,}")

print("\n✓ 모든 테스트 완료!")
